In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

IMPORTING ALL THE RELEVANT LIBRARIES
* MADE USE OF KERAS PROMINENTLY

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import pickle 
import shutil 
import matplotlib.pyplot as plt

In the above mentioned code we have used PICKLE AND SHUTIL which i have described below:
1. PICKLE - serializing and deserializing a Python object structure - coverting it into a byte stream - process called pickling
2.  SHUTIL - provides many functions of high-level operations on files and collections of files

In [ ]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split

In [ ]:
from keras.utils import np_utils
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

In [ ]:
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint
from keras.applications.vgg16 import VGG16


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

PYTHON IMAGING LIBRARY - PIL, IT ADDS IMAGE PROCESSING TO PYTHON

In [ ]:

from PIL import ImageFile 
from tqdm import tqdm

Below we have used the OS MODULE - it helps in connecting/interacting  with the operating system.
os.path.join() that we have used below is a method in Python  which is used to join one or more path components intelligently.










In [ ]:
datalink = "../input/state-farm-distracted-driver-detection/imgs"
test_data = os.path.join(datalink,"test")
train_data = os.path.join(datalink,"train")

In [ ]:
csvlink = os.path.join(os.getcwd(),"csv_files")
modelpath = os.path.join(os.getcwd(),"model","vgg16")
picklepath = os.path.join(os.getcwd(),"pickle")
testcsv = os.path.join(os.getcwd(),"csv_files","test.csv")
traincsv = os.path.join(os.getcwd(),"csv_files","train.csv")

In [ ]:
if not os.path.exists(test_data):
    print("Testing data -> does not exists")
if not os.path.exists(train_data):
    print("Training data -> does not exists")
if not os.path.exists(modelpath):
    print("Model path -> does not exists")
    os.makedirs(modelpath)
    print("Model path created")
    
else:
    shutil.rmtree(modelpath)
    os.makedirs(modelpath)
if not os.path.exists(picklepath):
    os.makedirs(picklepath)
if not os.path.exists(csvlink):
    os.makedirs(csvlink)


In [ ]:
#LENGTH
print(len(train_data))
print(len(traincsv))

print(len(test_data))
print(len(testcsv))

path. isdir() -> method in Python is used to check whether the specified path is an existing directory or not. If a path has a symbolic link linking to a directory it will return "TRUE"

In [ ]:
def csvfunc(datalink,filename):
    class_names = os.listdir(datalink)
    data = list()
    if(os.path.isdir(os.path.join(datalink,class_names[0]))):
        for class_name in class_names:
            file_names = os.listdir(os.path.join(datalink,class_name))
            for file in file_names:
                data.append({
                    "Filename":os.path.join(datalink,class_name,file),
                    "ClassName":class_name
                })
    else:
        class_name = "test"
        file_names = os.listdir(datalink)
        for file in file_names:
            data.append(({
                "FileName":os.path.join(datalink,file),
                "ClassName":class_name
            }))
    data = pd.DataFrame(data)
    data.to_csv(os.path.join(os.getcwd(),"csv_files",filename),index=False)

csvfunc(train_data,"train.csv")
csvfunc(test_data,"test.csv")
data_train = pd.read_csv(os.path.join(os.getcwd(),"csv_files","train.csv"))
data_test = pd.read_csv(os.path.join(os.getcwd(),"csv_files","test.csv"))

Details of the images are mentioned below 

In [ ]:
imgdetails = '../input/state-farm-distracted-driver-detection/'
driver_imgs_list = pd.read_csv(os.path.join(imgdetails,'driver_imgs_list.csv'))
driver_imgs_list.head()

In [ ]:
data_train = pd.read_csv(traincsv)
data_test = pd.read_csv(testcsv)

In [ ]:
#visual representation
driver_imgs_list.groupby('classname')['img'].count().sort_values().plot(kind='box')
plt.ylabel('Images range')
plt.show()

In [ ]:
from keras import *
from keras.utils.np_utils import to_categorical


labels_list = list(set(data_train['ClassName'].values.tolist()))
labels_id = {label_name:id for id,label_name in enumerate(labels_list)}
print(labels_id)
data_train['ClassName'].replace(labels_id,inplace=True)

labels = to_categorical(data_train['ClassName'])
print(labels.shape)

with open(os.path.join(picklepath,"labels_list_vgg16.pkl"),"wb") as handle:
    pickle.dump(labels_id,handle)

Below we have printed out the size of each image

In [ ]:
from PIL import Image 
img = Image.open('../input/state-farm-distracted-driver-detection/imgs/test/img_1.jpg')
print(img.size)

* load_img - loads RGB image as PIL.Image type
* img_to_array - converts tge PIL image type to 3d tensor with a shape.
*  expand_dims -  convert 3D tensor to 4D tensor with shape (1, 64, 64, 3) and return 4D tensor

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(data_train.iloc[:,0],labels,test_size = 0.2,random_state=42)
def path_to_tensor(img_path):
    
    img = image.load_img(img_path, target_size=(64, 64))
    
    x = image.img_to_array(img)

    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

Pre-Process the data for Keras

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

train_tensors = paths_to_tensor(xtrain).astype('float32')/255 - 0.5
valid_tensors = paths_to_tensor(xtest).astype('float32')/255 - 0.5

In [ ]:
model = VGG16(include_top=False)
model.summary()

In [ ]:
model = VGG16(include_top=False)
train_vgg16 = model.predict(train_tensors,verbose=1)
valid_vgg16 = model.predict(valid_tensors,verbose=1)
train_features = train_vgg16[0]
valid_features = valid_vgg16[0]

In [ ]:


VGG16_model = Sequential()
VGG16_model.add(GlobalAveragePooling2D(input_shape=train_features.shape))
VGG16_model.add(Dense(10, activation='softmax', kernel_initializer='glorot_normal'))
VGG16_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
filepath = os.path.join(modelpath,"distracted-{epoch:02d}-{val_accuracy:.2f}.hdf5")
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max',period=1)
callbacks_list = [checkpoint]

In [ ]:
model_history = VGG16_model.fit(train_vgg16,ytrain,validation_data = (valid_vgg16, ytest),epochs=200, batch_size=16, shuffle=True,callbacks=callbacks_list)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(model_history.history['loss'], color='b', label="Training loss")
ax1.plot(model_history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, 400, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(model_history.history['accuracy'], color='b', label="Training accuracy")
ax2.plot(model_history.history['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, 400, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()


In [ ]:
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    fig.savefig(os.path.join(modelpath,"confusion_matrix.png"))
    return fig

In [ ]:
def print_heatmap(n_labels, n_predictions, class_names):
    labels = n_labels 
    predictions = n_predictions 

    matrix = confusion_matrix(labels.argmax(axis=1),predictions.argmax(axis=1))
    row_sum = np.sum(matrix, axis = 1)
    w, h = matrix.shape

    c_m = np.zeros((w, h))

    for i in range(h):
        c_m[i] = matrix[i] * 100 / row_sum[i]

    c = c_m.astype(dtype = np.uint8)

    
    heatmap = print_confusion_matrix(c, class_names, figsize=(18,10), fontsize=20)

In [ ]:
class_names = list()
for name,idx in labels_id.items():
    class_names.append(name)
# print(class_names)
ypred = VGG16_model.predict(valid_vgg16,verbose=1)

Visual representation using heatmap

In [ ]:
print_heatmap(ytest,ypred,class_names)


In [ ]:
ypred_class = np.argmax(ypred,axis=1)
ytest = np.argmax(ytest,axis=1)

FORMULAS OF PRECISION. RECALL
* precision tp / (tp + fp)
* recall: tp / (tp + fn)
* f1: 2 tp / (2 tp + fp + fn)

In [ ]:
accuracy = accuracy_score(ytest,ypred_class)
print('Accuracy -  %f' % accuracy)

precision = precision_score(ytest, ypred_class,average='weighted')
print('Precision -  %f' % precision)

recall = recall_score(ytest,ypred_class,average='weighted')
print('Recall -  %f' % recall)

f1 = f1_score(ytest,ypred_class,average='weighted')
print('F1 score -  %f' % f1)